<div align="center">

# Getting Started with Patra Model Card Toolkit

</div>

The Patra Toolkit is a component of the Patra ModelCards framework designed to simplify the process of creating and documenting AI/ML models. It provides a structured schema that guides users in providing essential information about their models, including details about the model's purpose, development process, and performance. The toolkit also includes features for semi-automating the capture of key information, such as fairness and explainability metrics, through integrated analysis tools. By reducing the manual effort involved in creating model cards, the Patra Toolkit encourages researchers and developers to adopt best practices for documenting their models, ultimately contributing to greater transparency and accountability in AI/ML development.

---

## Features

1. **Encourages Accountability**
   - Incorporate essential model information (metadata, dataset details, fairness, explainability) at training time, ensuring AI models remain transparent from development to deployment.

2. **Semi-Automated Capture**
   - Automated *Fairness* and *Explainability* scanners compute demographic parity, equal odds, SHAP-based feature importances, etc., for easy integration into Model Cards.

3. **Machine-Actionable Model Cards**
   - Produce a structured JSON representation for ingestion into the Patra Knowledge Base. Ideal for advanced queries on model selection, provenance, versioning, or auditing.

4. **Flexible Repository Support**
   - Pluggable backends for storing models/artifacts on **Hugging Face** or **GitHub**, unifying the model publishing workflow.

5. **Versioning & Model Relationship Tracking**
   - Maintain multiple versions of a model with recognized edges (e.g., `revisionOf`, `alternateOf`) using embedding-based similarity. This ensures clear lineages and easy forward/backward provenance.


---

This notebook demonstrates:

1. **Loading & Preprocessing** the UCI Adult Dataset  
2. **Training** a simple TensorFlow model  
3. **Creating a Model Card** with optional Fairness and XAI scans  
4. **Submitting** the Model Card (and optionally the model, inference label, and artifacts) to:
   - **Patra server** (for model card storage)  
   - **Backend** (Hugging Face or GitHub) for model storage

---

## 1. Environment Setup

In [ ]:
!pip install patra_toolkit

In [ ]:
!pip install tensorflow

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("absl").setLevel(logging.ERROR)
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)
logging.getLogger("PyGithub").setLevel(logging.ERROR)

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import requests, io

from patra_toolkit import ModelCard, AIModel

## 2. Load and Pre-process the Data

We'll use the **UCI Adult Dataset**, which predicts whether an individual's income is above or below $50K based on demographics. This dataset is a common benchmark for exploring model fairness.

In [ ]:
resp = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")
resp.raise_for_status()

cols = [
    "age","workclass","fnlwgt","education","education_num",
    "marital_status","occupation","relationship","race",
    "sex","capital_gain","capital_loss","hours_per_week",
    "native_country","income"
]
df = pd.read_csv(io.StringIO(resp.text), names=cols, header=None)

# Encode target
df["income"] = LabelEncoder().fit_transform(df["income"])  # 1 if >50K, else 0

# One-hot encode everything except the target
df = pd.get_dummies(df, drop_first=True, dtype=float)

# Split into features/labels
X = df.drop("income", axis=1).astype("float32").values
y = df["income"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

Train shape: (26048, 100) Test shape: (6513, 100)


## 3. Train a Simple TensorFlow Model

Below is a straightforward neural network: two hidden layers plus a final sigmoid for binary classification. We'll train for a few epochs to demonstrate end-to-end usage.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/5
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6514 - loss: 425.1485
Epoch 2/5
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6807 - loss: 45.0486
Epoch 3/5
407/407 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - accuracy: 0.6847 - loss: 55.8134
Epoch 4/5
407/407 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - accuracy: 0.6852 - loss: 31.7177
Epoch 5/5
407/407 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - accuracy: 0.6801 - loss: 45.6668
Test Loss: 12.5695, Test Accuracy: 0.7857


## 4. Building a Patra Model Card

### 4.1 Basic Model Card Setup
We start with essential metadata like name, version, short description, and so on.  

In [ ]:
mc = ModelCard(
    name="UCI_Adult_Model",
    version="1.0",
    short_description="Predicting whether an individual's income is above $50K using TensorFlow.",
    full_description=(
        "This is a feed-forward neural network trained on the UCI Adult Dataset. "
        "It demonstrates how Patra Toolkit can store model details, fairness scans, "
        "and basic explainability data in a comprehensive Model Card."
    ),
    keywords="uci, adult, patra, fairness, xai, tensorflow",
    author="neelk",
    input_type="Tabular",
    category="classification",
    citation="Becker, B. & Kohavi, R. (1996). Adult [Dataset]. UCI."
)

### 4.2 Attach AI Model Information
Here we describe the model's ownership, license, performance metrics, etc.

In [ ]:
ai_model = AIModel(
    name="AdultTFModel",
    version="1.0",
    description="DNN on UCI Adult dataset for income prediction",
    owner="username",
    location="",
    license="BSD-3-Clause",
    framework="tensorflow",
    model_type="dnn",
    test_accuracy=accuracy
)

# Add additional performance or training metrics
ai_model.add_metric("Epochs", 5)
ai_model.add_metric("BatchSize", 64)
ai_model.add_metric("Optimizer", "Adam")

mc.ai_model = ai_model

## 5. Fairness & Explainability

### 5.1 Bias (Fairness) Analysis
Patra Toolkit has a built-in `populate_bias` method to measure metrics like **demographic parity** or **equalized odds**. We'll focus on the protected attribute "sex" in the data.

**Why check bias?** Real-world models often inadvertently penalize certain groups. By calling `mc.populate_bias(...)`, you get a quick sense of whether the model is systematically advantaging or disadvantaging certain subpopulations.

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()

mc.populate_bias(
    X_test,
    y_test,
    y_pred,
    "gender",           # Name you want displayed in the report
    X_test[:, 58],      # The slice of data that corresponds to gender
    model
)

print("Bias Analysis:\n", mc.bias_analysis)


204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 308us/step
Bias Analysis:
 {'demographic_parity_diff': 0.08162253952657954, 'equal_odds_difference': 0.08368136415250488}


### 5.2 Explainability (XAI)

If we want to understand model decisions, we can generate interpretability metrics (like feature importance) using Patra’s internal SHAP-based approach.

In [ ]:
# Rebuild the list of columns used in training
x_columns = df.columns.tolist()
x_columns.remove('income')  # Remove the target

mc.populate_xai(
    X_test[:10],
    x_columns,
    model
)

print("Explainability Analysis:\n", mc.xai_analysis)

Explainability Analysis:
 {'capital_gain': 0.4196902715030088, 'fnlwgt': 0.0010686394701219777, 'relationship__Wife': 0.0004169165563382081, 'occupation__Exec_managerial': 0.00037468428402272127, 'hours_per_week': 0.00030803950899628693, 'age': 0.0002882660328522926, 'education__HS_grad': 0.00017709857654685587, 'education__Masters': 0.00015673624516589788, 'occupation__Prof_specialty': 0.00012275671500096725, 'marital_status__Married_civ_spouse': 0.00010901905207295246}


## 6. Add Requirements and Validate
We let Patra auto-detect Python package dependencies to ensure reproducibility and then validate the card for completeness.

In [ ]:
mc.populate_requirements()
if mc.validate():
    print("Model Card is valid and ready to submit!")
else:
    print("Validation failed. See logs for details.")

INFO:root:Model card validated successfully.


Model Card is valid and ready to submit!


## 7. Submission Options

The `mc.submit(...)` method can do one or more of the following:
1. **Submit only the card** (no model, no artifacts).
2. **Include the trained model** (uploading to Hugging Face or GitHub).
3. **Add artifacts** (like data files, inference labels, or any additional resources).

Below, we demonstrate multiple usage patterns.

### 7.1 Submit **Only** the Model Card

No model, no inference label, no artifacts. Just the card is posted to your Patra server for cataloging.

In [ ]:
mc.submit(patra_server_url="http://127.0.0.1:5002")

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-uci_adult_model-1.0
INFO:root:Model Card submitted successfully.


'success'

### 7.2 Submit Model Card and Model

We can specify `"huggingface"` or `"github"` for `model_store`. This will attempt to upload our trained model, while the card is posted to the Patra server.

In [ ]:
mc.submit(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="h5",
    model_store="huggingface"
)

INFO:root:Model card validated successfully.
ERROR:root:Model submission failed during model ID creation: Model ID already exists. Please update the model version.


In [ ]:
mc.version = "1.1"
mc.submit(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="h5",
    model_store="huggingface"
)

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-uci_adult_model-1.1
INFO:root:Model serialized successfully.
INFO:root:Package 'huggingface_hub' not found. Installing...



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
neelk-uci_adult_model-1.1.h5: 100%|██████████| 130k/130k [00:00<00:00, 432kB/s]
INFO:root:Model uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.1/blob/main/neelk-uci_adult_model-1.1.h5
INFO:root:Model Card submitted successfully.


'success'

### 7.3 Submit Artifacts

In [ ]:
mc.submit(
    patra_server_url="http://127.0.0.1:5002",
    model_store="huggingface",
    artifacts=["data/adult/adult.data",
               "data/adult/adult.names",
               "data/adult/adult.names"]
)

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-uci_adult_model-1.1
INFO:root:Artifact 'data/adult/adult.data' uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.1/blob/main/adult.data
INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.1/blob/main/adult.names
No files have been modified since last commit. Skipping to prevent empty commit.
INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.1/blob/main/adult.names
INFO:root:Model Card submitted successfully.


'success'

### 7.4 Submit Model Card, Model, and Artifacts

This scenario might include a special label file plus multiple dataset artifacts.

In [ ]:
mc.submit(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="h5",
    model_store="huggingface",
    inference_labels="data/labels.txt",
    artifacts=["data/adult/adult.data",
               "data/adult/adult.names",
               "data/adult/adult.names"]
)

INFO:root:Model card validated successfully.
ERROR:root:Model submission failed during model ID creation: Model ID already exists. Please update the model version.


In [ ]:
mc.version = "1.2"
mc.submit(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="h5",
    model_store="huggingface",
    inference_labels="data/labels.txt",
    artifacts=["data/adult/adult.data",
               "data/adult/adult.names",
               "data/adult/adult.names"]
)

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-uci_adult_model-1.2
INFO:root:Model serialized successfully.
neelk-uci_adult_model-1.2.h5: 100%|██████████| 130k/130k [00:00<00:00, 1.08MB/s]
INFO:root:Model uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.2/blob/main/neelk-uci_adult_model-1.2.h5
INFO:root:Inference label uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.2/blob/main/labels.txt
INFO:root:Artifact 'data/adult/adult.data' uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.2/blob/main/adult.data
INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.2/blob/main/adult.names
No files have been modified since last commit. Skipping to prevent empty commit.
INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://huggingface.co/patra-iu/neelk-uci_adult_model-1.2/blob/main/adult.names
INFO:root:Model Card submitted successfully

'success'

### 7.4 Pushing to GitHub

By switching `"huggingface"` to `"github"`, you can store your model in a GitHub repo.

In [ ]:
mc.version = "1.3"
mc.submit(
    patra_server_url="http://127.0.0.1:5002",
    model=model,
    file_format="h5",
    model_store="github",
    artifacts=["data/adult/adult.data",
               "data/adult/adult.names",
               "data/adult/adult.names"]
)

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-uci_adult_model-1.3
INFO:root:Model serialized successfully.
INFO:root:Package 'PyGithub' not found. Installing...


Repository 'neelk-uci_adult_model-1.3' already exists. Using existing repository.


INFO:root:Model uploaded at: https://github.com/nee1k/neelk-uci_adult_model-1.3/blob/main/neelk-uci_adult_model-1.3.h5


Repository 'neelk-uci_adult_model-1.3' already exists. Using existing repository.
No changes to commit, skipping commit step.


INFO:root:Inference label uploaded at: https://github.com/nee1k/neelk-uci_adult_model-1.3/blob/main/labels.txt


...

Repository 'neelk-uci_adult_model-1.3' already exists. Using existing repository.
No changes to commit, skipping commit step.


INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://github.com/nee1k/neelk-uci_adult_model-1.3/blob/main/adult.names


Repository 'neelk-uci_adult_model-1.3' already exists. Using existing repository.
No changes to commit, skipping commit step.


INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://github.com/nee1k/neelk-uci_adult_model-1.3/blob/main/adult.names
INFO:root:Model Card submitted successfully.


'success'

By following this notebook, you have:
1. Loaded and preprocessed the UCI Adult Dataset
2. Trained a TensorFlow model to predict income
3. Built a Patra Model Card describing the model’s purpose, performance, and environment
4. Scanned for fairness and explainability metrics
5. Submitted the card to a Patra server along with the model or artifacts to a chosen store (Hugging Face or GitHub)
